In [1]:
import os

os.chdir('../')
%pwd

'/home/emmanuel/Desktop/Dynamic-Pricing-Engine'

In [2]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor

In [3]:
xbg = XGBRegressor(n_job=-1)

In [ ]:
from dateutil.relativedelta import relativedelta
from datetime import timedelta, datetime
#extracting the date
datetime.today().strftime('%Y-%m-%d')
#datetime.today()# - relativedelta(months=1))

SyntaxError: invalid syntax (2942635986.py, line 4)

In [2]:
from src.DynamicPricingEngine.utils.data_ingestion_utils import time_subtract

In [11]:
## two months into the past
now = (datetime.today()- relativedelta(months=1))
# end_date = datetime.strptime(now, "%Y-%m-%d") ## converting to datetime
end_date = now - timedelta(days=now.day) ## retrieving the last day of the previous month

## accessing the previous month
days_to_subtract = time_subtract(end_date.strftime('%Y-%m-%d'))
end_date = (end_date - timedelta(days=days_to_subtract))

## start of the month
days= time_subtract(end_date.strftime('%Y-%m-%d'))
start_date= end_date - timedelta(days=days-1)

#self.config = config
start_date = start_date.strftime('%Y-%m-%d')
end_date = end_date.strftime('%Y-%m-%d')

print(f'start date: {start_date}, end date: {end_date}')

start date: 2025-10-01, end date: 2025-10-31


## Exploratory Data Anaysis

### Things to do:
- Data Collection
- Data Checks
- Exploratory Data Analysis
- Data Preproccessing and Feature Engineering
- Model Training, Evaluation, and Hyperparameter Tuning

In [5]:
## importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

[ 2026-01-17 14:52:16,436 ]: matplotlib.font_manager: INFO: font_manager: 1107: Failed to extract font properties from /usr/share/fonts/noto/NotoColorEmoji.ttf: Can not load face (unknown file format; error code 0x2)
[ 2026-01-17 14:52:16,542 ]: matplotlib.font_manager: INFO: font_manager: 1639: generated new fontManager


In [ ]:
df_html = pd.read_html('https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)',match = 'by country')
# Let's see how many tables are there with tage ' by county'
print(len(df_html)) # There are 4 tables
# Let's see the first table
df_html[0]

### Extracting Weather Information for NYC

In [ ]:
#from pyspark.sql import SparkSession

spark = ('dynamicpricing').getOrCreate()
#spark = (
 #   SparkSession.builder
  #      .master("local")
   #     .appName("dynamicPricing")
    #    .getOrCreate())

In [ ]:
#os.environ['JAVA_HOME']= 

In [ ]:
os.environ['SPARK_HOME']= 'opt/apache-spark/'

In [6]:
import os, sys, pickle
import pandas as pd
import numpy as np
import geopandas as gpd
import dask.dataframe as dd
from datetime import datetime, timedelta
import hopsworks
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

load_dotenv()

from src.DynamicPricingEngine.logger.logger import logger
from src.DynamicPricingEngine.exception.customexception import RideDemandException
from src.DynamicPricingEngine.entity.config_entity import DataTransformationConfig
from src.DynamicPricingEngine.utils.common_utils import load_shapefile_from_zip
from src.DynamicPricingEngine.utils.data_ingestion_utils import time_subtract


class DataTransformation:
    def __init__(self, config: DataTransformationConfig, 
                 nyc_taxi_data: str, 
                 nyc_weather_data: str
                 ):
        
        self.config = config
        
        #Read both datasets with Dask
        self.taxi_df  =dd.read_parquet(nyc_taxi_data, engine='pyarrow')
        self.weather_df =dd.read_csv(nyc_weather_data)

        self.taxi_df.index = self.taxi_df.index.astype('int32')
        self.weather_df.index = self.weather_df.index.astype('int32')

        #Ensure datetime types
        for col in ['tpep_pickup_datetime', 'tpep_dropoff_datetime']:
            if col in self.taxi_df.columns:
                self.taxi_df[col] = dd.to_datetime(self.taxi_df[col], errors='coerce')

        # Precompute bin
        if 'tpep_pickup_datetime' in self.taxi_df.columns:
            self.taxi_df['bin'] = self.taxi_df['tpep_pickup_datetime'].dt.floor('60min')

         #Cache neighbor dictionary
        self._neighbor_dict = None
        self._neighbor_cache_path = os.path.join(self.config.shapefile_dir, "neighbors.pkl")

    def _get_neighbor_dict(self) -> dict:
        if self._neighbor_dict is not None:
            return self._neighbor_dict

        if os.path.exists(self._neighbor_cache_path):
            try:
                with open(self._neighbor_cache_path, "rb") as f:
                    self._neighbor_dict = pickle.load(f)
                logger.info("Loaded neighbor dictionary from cache")
                return self._neighbor_dict
            except Exception as e:
                logger.warning(f"Failed to load neighbor cache: {e}")

        zones_gdf = load_shapefile_from_zip(self.config.taxi_zone_shapefile_url, 
                                            self.config.shapefile_dir)
        zones_gdf_left = zones_gdf.rename(columns={"LocationID": "LocationID_left"})
        zones_gdf_right = zones_gdf.rename(columns={"LocationID": "LocationID_right"})
        neighbors_df = gpd.sjoin(zones_gdf_left, zones_gdf_right, how="left", predicate="touches")
        neighbors_df = neighbors_df[neighbors_df['LocationID_left'] != neighbors_df['LocationID_right']]
        self._neighbor_dict = (neighbors_df.groupby('LocationID_left')['LocationID_right']
                               .apply(lambda s: sorted(list(set(s))))
                               .to_dict())
        try:
            os.makedirs(self.config.shapefile_dir, exist_ok=True)
            with open(self._neighbor_cache_path, "wb") as f:
                pickle.dump(self._neighbor_dict, f)
        except Exception as e:
            logger.warning(f"Failed to persist neighbor cache: {e}")
        return self._neighbor_dict


    def derive_target_and_join_to_weather_feature(self) -> dd.DataFrame:
        try:
            taxi_df = self.taxi_df[['PULocationID', 'bin']]

            # Aggregate pickups per zone-hour
            y = (taxi_df
                 .groupby(['PULocationID', 'bin'])
                 .size().rename('pickups')
                 .reset_index())
            
            y['bin']= y['bin'].astype('datetime64[ns]')
            y['PULocationID']= y['PULocationID'].astype('int32')

            # Materialized data to Pandas to build full grid then back to Dask
            zones = y['PULocationID'].unique().compute()

            time_index = pd.date_range(y['bin'].min().compute(), 
                                       y['bin'].max().compute(), 
                                       freq='60min')
            
            grid = pd.MultiIndex.from_product([zones, time_index], 
                                              names=['PULocationID', 
                                                     'bin']).to_frame(index=False)

            #y['PULocationID'] = y['PULocationID'].astype('int32')
            grid['PULocationID'] = grid['PULocationID'].astype('int32')

            # Align datetime precision
            y['bin'] = y['bin'].astype('datetime64[ns]')
            grid['bin'] = grid['bin'].astype('datetime64[ns]')

            y = dd.from_pandas(grid, npartitions=4).merge(y, how='left', 
                                                          on=['PULocationID', 'bin'])
            y = y.fillna({'pickups': 0})

            # Weather alignment
            weather_df = self.weather_df
            weather_df['bin'] = dd.to_datetime(
                weather_df['day'].astype(str) + ' ' + 
                weather_df['datetime'].astype(str),
                errors='coerce'
            )
            
            #Dropping the Day column
            weather_df = weather_df.drop(columns='day')
            y.index = y.index.astype('int32')

            # Merge target + weather and sort by PUlocationID and bin
            df = y.merge(weather_df, on='bin', how='left').map_partitions(
                lambda pdf: pdf.sort_values(['PULocationID', 'bin'])
            )

            return (df)

        except Exception as e:
            logger.error("Failed to generate the target feature", exc_info=True)
            raise RideDemandException(e, sys)
        
    def engineer_temporal_feature(self, df: dd.DataFrame) -> dd.DataFrame:
        """
        Deriving the temporal feature
        
        :type df: dd.DataFrame
        :return: Description
        :rtype: DataFrame
        """
        try:
            # Temporal from 'bin'
            df['pickup_year'] = df['bin'].dt.year
            df['pickup_month'] = df['bin'].dt.month
            df['day_of_month'] = df['bin'].dt.day
            df['Pickup_hour'] = df['bin'].dt.hour
            df['day_of_week'] = df['bin'].dt.dayofweek
            df["bin_str"] = df["bin"].astype('str')

            # Vectorized flags
            df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype('int8')
            df['is_rush_hour'] = df['Pickup_hour'].isin([7, 8, 9, 16, 17, 18, 19]).astype('int8')
            df['is_night_hour'] = df['Pickup_hour'].isin([0,1,2,3,4,5,6,20,21,22,23]).astype('int8')

            # Season mapping
            season_map = {
                12: 'winter', 1: 'winter', 2: 'winter',
                3: 'spring', 4: 'spring', 5: 'spring',
                6: 'summer', 7: 'summer', 8: 'summer',
                9: 'autumn', 10: 'autumn', 11: 'autumn'
            }
            df['season_of_year'] = df['pickup_month'].map(season_map)

            # Fixed holidays and specials
            fixed_holidays = {(1, 1), (7, 4), (11, 11), (6, 19), (12, 25)}
            fixed_specials = {(3, 17), (7, 4), (6, 4), (12, 31),
                            (6, 5), (6, 6), (6, 7), (6, 8), (6, 9), (6, 10),
                            (6, 11), (6, 12), (6, 13), (6, 14), (6, 15)}

            df['is_holiday'] = df[['pickup_month', 'day_of_month']].map_partitions(
                lambda pdf: pdf.apply(lambda r: int((r['pickup_month'], 
                                                     r['day_of_month']) in fixed_holidays), axis=1),
                meta=('is_holiday', 'int8')
            )

            df['Is_special_event'] = df[['pickup_month', 'day_of_month']].map_partitions(
                lambda pdf: pdf.apply(lambda r: int((r['pickup_month'], 
                                                     r['day_of_month']) in fixed_specials), axis=1),
                meta=('Is_special_event', 'int8')
            )

            # Movable holidays and specials (row-wise logic via map_partitions)
            def add_movable_flags(pdf: dd.DataFrame) -> dd.DataFrame:
                def _movable_holiday(row):
                    y, m, d = row['pickup_year'], row['pickup_month'], row['day_of_month']
                    date = pd.Timestamp(y, m, d)
                    if m == 11 and date.weekday() == 3 and date + pd.Timedelta(days=7) > pd.Timestamp(y, 11, 30):
                        return 1
                    if m == 5 and date.weekday() == 0 and date + pd.Timedelta(days=7) > pd.Timestamp(y, 5, 31):
                        return 1
                    if m == 1 and date.weekday() == 0 and 15 <= d <= 21:
                        return 1
                    if m == 9 and date.weekday() == 0 and 1 <= d <= 7:
                        return 1
                    if m == 11 and date.weekday() == 1 and 1 <= d <= 7:
                        return 1
                    if m == 10 and date.weekday() == 0 and 8 <= d <= 14:
                        return 1
                    return 0

                def _movable_special(row):
                    y, m, d = row['pickup_year'], row['pickup_month'], row['day_of_month']
                    date = pd.Timestamp(y, m, d)
                    if m == 11 and date.weekday() == 3 and date + pd.Timedelta(days=7) > pd.Timestamp(y, 11, 30):
                        return 1
                    if m == 6 and date.weekday() == 6 and date + pd.Timedelta(days=7) > pd.Timestamp(y, 6, 30):
                        return 1
                    return 0

                pdf['is_holiday'] = pdf['is_holiday'].where(pdf['is_holiday'] == 1,
                                                        pdf.apply(_movable_holiday, axis=1))
                pdf.loc[pdf['Is_special_event'] == 0, 'Is_special_event'] = pdf.loc[pdf['Is_special_event'] == 0].apply(_movable_special, axis=1)
                
                return pdf

            df = df.map_partitions(add_movable_flags) #meta=df._meta)

            ## creating a column for Payday Indicator
            def is_payday(data):

                date = pd.Timestamp(year=data['pickup_year'], 
                                    month=data['pickup_month'], 
                                    day=data['day_of_month'])

                if date.is_month_end:
                    return 1
                
                if date.day ==(15 or 16 or 17) and date.isoweekday!=(6 or 7):
                    return 1

                return 0

            #pdf = df.compute()

            pdf= df[['PULocationID', 'bin', 'pickup_year', 
                     'pickup_month', 'day_of_month']].compute()

            pdf['is_payday'] = pdf.apply(is_payday, axis=1) ##deriving the payday indicator

            df = df.merge(dd.from_pandas(pdf, npartitions=4), 
                          on=['PULocationID', "bin"], how='left')
            df= df.rename(columns={'pickup_year_x':'pickup_year', 
                                   'pickup_month_x':'pickup_month', 
                                   'day_of_month_x':'day_of_month'})
            
            df = df.drop(['pickup_year_y', 'pickup_month_y', 
                          'day_of_month_y'], axis=1)

            return df

        except Exception as e:
            logger.error("Failed to engineer temporal features", exc_info=True)
            raise RideDemandException(e, sys)


    def city_wide_congestion_features(self, df: pd.DataFrame) -> dd.DataFrame:
        try:
            # Select needed columns
            taxi_df = self.taxi_df[['bin', 'tpep_pickup_datetime', 
                                    'tpep_dropoff_datetime', 'trip_distance']]

            # Compute trip duration in hours
            taxi_df['trip_duration_hr'] = (
                (taxi_df['tpep_dropoff_datetime'] - 
                 taxi_df['tpep_pickup_datetime']).dt.total_seconds() / 3600
            )

            # Filter invalid trips
            taxi_df = taxi_df[(taxi_df['trip_duration_hr'] > 0) & 
                              (taxi_df['trip_distance'] >= 0)]

            # Compute speed
            taxi_df['MPH'] = taxi_df['trip_distance'] / taxi_df['trip_duration_hr']
            taxi_df = taxi_df[taxi_df['MPH'].between(1, 60)]

            # Compute citywide average speed per hour
            city_speed = (
                taxi_df.groupby('bin')['MPH']
                .mean()
                .reset_index()
                .rename(columns={'MPH': 'city_avg_speed'})
            )

            # Congestion index
            city_speed['city_congestion_index'] = city_speed['city_avg_speed'].map_partitions(
                lambda pdf: np.where(pdf > 0, 1.0 / pdf, np.nan),
                meta=('city_congestion_index', 'f8')
            )

            # Merge back into main df
            df = df.merge(city_speed, on='bin', how='left')
            return df

        except Exception as e:
            logger.error("Unable to generate city-wide features", exc_info=True)
            raise RideDemandException(e, sys)


    def zone_level_congestion_features(self, df: dd.DataFrame) -> dd.DataFrame:
        try:
            # Select needed columns
            taxi_df = self.taxi_df[['PULocationID', 'bin', 'tpep_pickup_datetime',
                                    'tpep_dropoff_datetime', 'trip_distance']]

            # Compute trip duration in hours
            taxi_df['trip_duration_hr'] = (
                (taxi_df['tpep_dropoff_datetime'] 
                 -taxi_df['tpep_pickup_datetime'])
                 .dt.total_seconds() / 3600
            )

            # Filter invalid trips
            taxi_df = taxi_df[(taxi_df['trip_duration_hr'] > 0) & (taxi_df['trip_distance'] >= 0)]

            # Compute speed
            taxi_df['MPH'] = taxi_df['trip_distance'] / taxi_df['trip_duration_hr']
            taxi_df = taxi_df[taxi_df['MPH'].between(1, 60)]

            # Compute zone-level average speed per hour
            zone_speed = (
                taxi_df.groupby(['PULocationID', 'bin'])['MPH']
                .mean()
                .reset_index()
                .rename(columns={'MPH': 'zone_avg_speed'})
            )

            # Congestion index
            zone_speed['zone_congestion_index'] = zone_speed['zone_avg_speed'].map_partitions(
                lambda pdf: np.where(pdf > 0, 1.0 / pdf, np.nan),
                meta=('zone_congestion_index', 'f8')
            )

            # Build full grid (zones × time) in Pandas, then convert back to Dask
            zones = df['PULocationID'].unique().compute()
            time_index = pd.date_range(df['bin'].min().compute(), 
                                       df['bin'].max().compute(), freq='60min'
                                       )
            
            grid = pd.MultiIndex.from_product([zones, time_index], 
                                              names=['PULocationID', 
                                                     'bin']).to_frame(index=False
                                                                      )
            
            grid_dd = dd.from_pandas(grid, npartitions=4)

            zone_speed['PULocationID']= zone_speed['PULocationID'].astype('int32')

            # Merge grid with zone_speed to ensure full coverage
            zone_speed = grid_dd.merge(zone_speed, how='left', on=['PULocationID', 'bin'])
            zone_speed[['zone_avg_speed', 'zone_congestion_index']] = zone_speed[
                ['zone_avg_speed', 'zone_congestion_index']
            ].fillna(0)

            #zone_speed['PULocationID']= zone_speed['PULocationID'].astype('int32')
            # Merge back into main df
            df = df.merge(zone_speed, on=['PULocationID', 'bin'], how='left')
            return df

        except Exception as e:
            logger.error("Unable to generate zone-level features", exc_info=True)
            raise RideDemandException(e, sys)


    def citywide_hourly_demand(self, df: dd.DataFrame) -> dd.DataFrame:
        try:
            # Aggregate citywide pickups per hour
            city_demand = (
                df.groupby('bin')['pickups']
                .sum()
                .reset_index()
                .rename(columns={'pickups': 'city_pickups'})
            )

            # Merge back into main df
            df = df.merge(city_demand, on='bin', how='left')

            return df

        except Exception as e:
            logger.error("Unable to engineer citywide hourly demand features", exc_info=True)
            raise RideDemandException(e, sys)


    def generate_neighbor_features(self, df: dd.DataFrame) -> dd.DataFrame:
        try:
            neighbor_dict = self._get_neighbor_dict()

            # Build neighbor pairs in Pandas, then convert to Dask
            neighbor_pdf = pd.DataFrame(
                [(zone, n) for zone, neighs in neighbor_dict.items() for n in neighs],
                columns=['PULocationID', 'neighbor_id']
            ).fillna(-1)

            neighbor_pdf['PULocationID'] = neighbor_pdf['PULocationID'].astype(
                df['PULocationID'].dtype
                )
            neighbor_pdf['neighbor_id'] = neighbor_pdf['neighbor_id'].astype(
                df['PULocationID'].dtype
                )

            neighbor_ddf = dd.from_pandas(neighbor_pdf, npartitions=1)

            # Prepare neighbor pickups
            df_neighbors = df[['PULocationID', 'bin', 'pickups']].rename(
                columns={'PULocationID': 'neighbor_id', 'pickups': 'neighbor_pickups'}
            )

            merged = neighbor_ddf.merge(df_neighbors, on='neighbor_id', how='left')

            neighbor_demand_df = (
                merged.groupby(['PULocationID', 'bin'])['neighbor_pickups']
                .sum()
                .reset_index()
                .rename(columns={'neighbor_pickups': 'neighbor_pickups_sum'})
            )

            neighbor_demand_df['neighbor_pickups_sum'] = neighbor_demand_df['neighbor_pickups_sum']#.fillna(-1)

            df = df.merge(neighbor_demand_df, on=['PULocationID', 'bin'], how='left')
            df['neighbor_pickups_sum'] = df['neighbor_pickups_sum'].fillna(0)

            return df

        except Exception as e:
            logger.error("Unable to generate neighbor features", exc_info=True)
            raise RideDemandException(e, sys)
        
    
    def engineer_autoregressive_signals(self, df: dd.DataFrame) -> dd.DataFrame:
        try:
            ## Define a Pandas function to apply per-partition
            pdf= df[['PULocationID', 'bin', 'pickups', 
                     'city_pickups', 'neighbor_pickups_sum']].compute()

            def make_lags(group, col='pickups'):

                ## for lag features
                for l in [1,24]:
                    group[f'{col}_lag_{l}h'] = group[col].shift(l)

                ## for rolling mean and std for zonelevel/bin
                for w in [24]:
                    group[f'{col}_roll_mean_{w}h'] = group[col].shift(1).rolling(w).mean()
                    group[f'{col}_roll_std_{w}h'] = group[col].shift(1).rolling(w).std()
                return group
            
            pdf.reset_index()
            pdf = pdf.sort_values(['PULocationID','bin'])

            ##generating the autoregressive feature
            pdf = pdf.groupby('PULocationID', 
                              group_keys=False).apply(make_lags) 

            # Create lag features for city pickups(1h, 24h)
            for lag in [1, 24]:
                pdf[f'city_pickups_lag_{lag}h'] = pdf['city_pickups'].shift(lag)

            ## computing the Lagged neighbor demand
            for lag in [1,24]:
                pdf[f'neighbor_pickups_lag_{lag}h'] = pdf.groupby(
                    'PULocationID')['neighbor_pickups_sum'].shift(lag)

            pdf.fillna(0, inplace=True)

            df = df.merge(dd.from_pandas(pdf, npartitions=4), on=['PULocationID', "bin"], how='left')
            df= df.rename(columns={'pickups_x':'pickups', 'city_pickups_x':'city_pickups', 
                       'neighbor_pickups_sum_x':'neighbor_pickups_sum'}
                       )
            
            df = df.drop(['pickups_y', 'city_pickups_y', 'neighbor_pickups_sum_y'], axis=1)

            return df

        except Exception as e:
            logger.error("Failed to generate autoregressive features", exc_info=True)
            raise RideDemandException(e, sys)

        
    def save_data_to_feature_store(self, df):
        try:
            transformed_data_store = self.config.transformed_data_file_path

            logger.info("Saving the transformed dataset to the feature store")
            os.makedirs(os.path.dirname(transformed_data_store), exist_ok=True)
            df.to_parquet(transformed_data_store)

            logger.info(f"Transformed data saved to path: {transformed_data_store}")
            print(f"Size of transformed data: {len(df)},{df.shape[1]}")

        except Exception as e:
            logger.error("Unable to save the file", exc_info=True)
            raise RideDemandException(e, sys)
        

    def push_transformed_data_to_feature_store(self, data)-> None:
        try:
            api = os.getenv('HOPSWORKS_API_KEY')
            
            ##initializing and login to hopswork feature store
            project = hopsworks.login(project='RideDemandPrediction', api_key_value=api)
            fs = project.get_feature_store()

            ## creating a new feature group
            fg = fs.get_or_create_feature_group(
                name = 'ridedemandprediction',
                version = 1,
                primary_key = ['PULocationID', 'bin_str'],
                event_time = 'bin',
                description = 'NYC yellow taxi pickup demands per hour per zone',
                online_enabled = False,
                partition_key = ['pickup_year','pickup_month']
            )

            ##converting dask dataframe to pandas dataframe
            data = data.compute()
            
            ## inserting new data in the feature group created above
            fg.insert(data, storage = 'offline', write_options = {'wait_for_job': True, 'use_spark':True})

            logger.info('data successfully added to hopsworks feature group')

        except Exception as e:
            raise RideDemandException(e,sys)

/home/emmanuel/Desktop/Dynamic-Pricing-Engine/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from src.DynamicPricingEngine.config.configuration import ConfigurationManager

In [41]:
taxi= 'artifacts/data_ingestion/taxi_data.parquet'
weather = 'artifacts/data_ingestion/weather_data.csv'

config = ConfigurationManager()
transform_config = config.get_data_transformation_config()

2026-01-09 20:02:59,986 INFO: successfully load the yaml file from path: config/config.yaml
2026-01-09 20:02:59,997 INFO: successfully load the yaml file from path: params.yaml
2026-01-09 20:02:59,998 INFO: successfully created directory at: ['artifacts']
2026-01-09 20:02:59,999 INFO: Artifacts root directory successfully created: artifacts
2026-01-09 20:03:00,000 INFO: successfully created directory at: ['artifacts/data_transformation']


In [ ]:
transform = DataTransformation(transform_config, taxi, weather)
df = transform.derive_target_and_join_to_weather_feature()
df = transform.engineer_temporal_feature(df)
df = transform.city_wide_congestion_features(df)
df = transform.zone_level_congestion_features(df)
df = transform.citywide_hourly_demand(df)
df = transform.generate_neighbor_features(df)
df = transform.engineer_autoregressive_signals(df)
#transform.save_data_to_feature_store(df)
transform.push_transformed_data_to_feature_store(df)



You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map function that you are using.
  Before: .map(func)
  After:  .map(func, meta=('pickup_month', 'object'))



2026-01-09 20:06:50,823 INFO: Loaded neighbor dictionary from cache
2026-01-09 20:07:01,970 INFO: Closing external client and cleaning up certificates.
Connection closed.
2026-01-09 20:07:01,973 INFO: Initializing external client
2026-01-09 20:07:01,974 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-09 20:07:03,063 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1257642


RideDemandException: There is an error in /tmp/ipykernel_27338/2067020572.py at line 521 with The truth value of a DataFrame is ambiguous. Use a.any() or a.all().

: 

In [38]:
transform.push_transformed_data_to_feature_store(df)

2026-01-09 20:01:43,881 INFO: Initializing external client
2026-01-09 20:01:43,884 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-09 20:01:46,785 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1257642


RideDemandException: There is an error in /tmp/ipykernel_27338/2152897444.py at line 521 with The truth value of a DataFrame is ambiguous. Use a.any() or a.all().

In [1]:
import pandas as pd

In [19]:
data = pd.read_parquet('artifacts/data_transformation/transformed_data.parquet')
data.head()

,PULocationID,bin,pickups,datetime,temp,dew,humidity,precip,snow,windspeed,...,neighbor_pickups_sum_x,pickups_lag_1h,pickups_lag_24h,pickups_roll_mean_24h,pickups_roll_std_24h,city_pickups_lag_1h,city_pickups_lag_24h,neighbor_pickups_lag_1h,neighbor_pickups_lag_24h,neighbor_pickups_sum
__null_dask_index__,,,,,,,,,,,,,,,,,,,,,
0,201,2025-11-01 00:00:00,1,00:00:00,53.5,28.6,38.14,0.0,0.0,13.6,...,0,0.0,0.0,0.0,0.0,3204.0,5066.0,0.0,0.0,0
1,201,2025-11-01 02:00:00,0,02:00:00,52.5,27.1,37.29,0.0,0.0,16.4,...,0,0.0,0.0,0.0,0.0,9349.0,2724.0,0.0,0.0,0
2,201,2025-11-01 05:00:00,0,05:00:00,50.4,27.1,40.29,0.0,0.0,12.1,...,0,0.0,0.0,0.0,0.0,5217.0,976.0,0.0,0.0,0
3,201,2025-11-01 15:00:00,0,15:00:00,57.2,29.5,34.57,0.0,0.0,14.0,...,0,1.0,0.0,0.0,0.0,8744.0,6907.0,0.0,0.0,0
4,201,2025-11-01 18:00:00,1,18:00:00,56.3,29.0,35.08,0.0,0.0,9.2,...,0,0.0,0.0,0.0,0.0,9825.0,6589.0,0.0,0.0,0


In [23]:
data.columns

Index(['PULocationID', 'bin', 'pickups', 'datetime', 'temp', 'dew', 'humidity',
       'precip', 'snow', 'windspeed', 'feelslike', 'snowdepth', 'visibility',
       'pickup_year', 'pickup_month', 'day_of_month', 'Pickup_hour',
       'day_of_week', 'bin_str', 'is_weekend', 'is_rush_hour', 'is_night_hour',
       'season_of_year', 'is_holiday', 'Is_special_event', 'is_payday',
       'city_avg_speed', 'city_congestion_index', 'zone_avg_speed',
       'zone_congestion_index', 'city_pickups', 'neighbor_pickups_sum_x',
       'pickups_lag_1h', 'pickups_lag_24h', 'pickups_roll_mean_24h',
       'pickups_roll_std_24h', 'city_pickups_lag_1h', 'city_pickups_lag_24h',
       'neighbor_pickups_lag_1h', 'neighbor_pickups_lag_24h',
       'neighbor_pickups_sum'],
      dtype='object')

In [22]:
data['Is_special_event'].dtype

dtype('int8')

In [21]:
#data['bin'] = pd.to_datetime(data['bin'])
data['pickups'].dtype

dtype('int64')

In [7]:
data['is_holiday'].dtype

dtype('int8')

In [14]:
data['neighbor_pickups_sum']

KeyError: 'neighbor_pickups_sum'

In [18]:
data.isnull().sum()

PULocationID                0
bin                         0
pickups                     0
datetime                    0
temp                        0
dew                         0
humidity                    0
precip                      0
snow                        0
windspeed                   0
feelslike                   0
snowdepth                   0
visibility                  0
pickup_year                 0
pickup_month                0
day_of_month                0
Pickup_hour                 0
day_of_week                 0
bin_str                     0
is_weekend                  0
is_rush_hour                0
is_night_hour               0
season_of_year              0
is_holiday                  0
Is_special_event            0
is_payday                   0
city_avg_speed              0
city_congestion_index       0
zone_avg_speed              0
zone_congestion_index       0
city_pickups                0
neighbor_pickups_sum_x      0
pickups_lag_1h              0
pickups_la

In [15]:
data.shape

(191100, 41)

In [ ]:

#@ensure_annotations
def is_holiday(df):
    # Weekends
    if df['day_of_week'] in [5, 6]:  # Saturday=5, Sunday=6
        return 1

    # Fixed-date holidays
    fixed_holidays = [(1, 1), (7, 4), (11, 11), (6, 19), (12, 25)]
    if (df['pickup_month'], df['day_of_month']) in fixed_holidays:
        return 1

    # Movable holidays
    date = pd.Timestamp(year=df['pickup_year'], month=df['pickup_month'], day=df['day_of_month'])

    # Last Thursday in November
    if df['pickup_month'] == 11 and date.weekday() == 3 and date + pd.Timedelta(
        days=7) > pd.Timestamp(year=df['pickup_year'], month=11, day=30):
        return 1

    # Last Monday in May
    if df['pickup_month'] == 5 and date.weekday() == 0 and date + pd.Timedelta(days=7) > pd.Timestamp(
        year=df['pickup_year'], month=5, day=31):
        return 1

    # Third Monday in January
    if df['pickup_month'] == 1 and date.weekday() == 0 and 15 <= df['day_of_month'] <= 21:
        return 1

    # First Monday in September
    if df['pickup_month'] == 9 and date.weekday() == 0 and 1 <= df['day_of_month'] <= 7:
        return 1

    # First Tuesday in November
    if df['pickup_month'] == 11 and date.weekday() == 1 and 1 <= df['day_of_month'] <= 7:
        return 1

    # Second Monday in October
    if df['pickup_month'] == 10 and date.weekday() == 0 and 8 <= df['day_of_month'] <= 14:
        return 1

    return 0

In [ ]:

import os,sys
import pandas as pd
import numpy as np
from datetime import datetime
import geopandas as gdp

from src.DynamicPricingEngine.logger.logger import logger
from src.DynamicPricingEngine.exception.customexception import RideDemandException
from src.DynamicPricingEngine.entity.config_entity import DataTransformationConfig
from src.DynamicPricingEngine.utils.common_utils import create_dir, load_shapefile_from_zip

class DataTransformation:
    def __init__(self, config:DataTransformationConfig, 
                 nyc_taxi_data:str,
                  nyc_weather_data:str):
        
        self.config = config
        taxi_data = pd.read_parquet(nyc_taxi_data)
        weather_data = pd.read_csv(nyc_weather_data)

        self.taxi_df = taxi_data
        self.weather_df = weather_data

    def derive_target_and_join_to_weather_feature(self)->pd.DataFrame:
        try:

            taxi_df = self.taxi_df
            weather_df = self.weather_df

            ## deriving the target feature
            taxi_df['bin'] = taxi_df['tpep_pickup_datetime'].dt.floor('60min')  ## bining into hour

            y = (taxi_df
                .groupby(['PULocationID','bin'])
                .size()
                .rename('pickups')
                .reset_index())

            # Build full grid to include zeros
            zones = y['PULocationID'].unique()
            time_index = pd.date_range(y['bin'].min(), y['bin'].max(), freq='60min')
            grid = pd.MultiIndex.from_product([zones, time_index], names=['PULocationID','bin']).to_frame(index=False)

            y = grid.merge(y, how='left', on=['PULocationID','bin']).fillna({'pickups':0})
            
            logger.info("target feature derived successfully")

            ## Adding the weather feature created to the already created pickups columns
            weather_df['bin'] = pd.to_datetime(weather_df['day'].astype(str) + ' ' + weather_df['datetime'].astype(str))

            #weather_hourly = (weather_df
             #               .groupby('bin')
              #              .agg({'temp':'mean','dew':'mean','snow':'sum',
               #                     'snowdepth':'sum',
                #                    'precip':'sum','windspeed':'mean',
                 #                   'visibility':'mean','humidity':'mean'})
                  #          .reset_index())

            df = y.merge(weather_df, on='bin', how='left').ffill()
            df.set_index('bin', inplace=True)
            logger.info("The data joined successfully")

            return df
        
        except Exception as e:
            logger.error("Failed to generate the target feature", e)
            raise RideDemandException(e,sys)
    
    def engineer_temporal_feature(self, df:pd.DataFrame)->pd.DataFrame:
        
        try:
            ##seperating tpep_pickup_datetime
            df['pickup_year']= df.index.year
            df['pickup_month']= df.index.month
            df['day_of_month']= df.index.day
            df['Pickup_hour']= df.index.hour
            df['day_of_week']= df.index.dayofweek

            ## creating the is_weekend, is_rush_hour, is_night_hour, is_holiday, season of the year, and special event data
            df['is_weekend'] = df['day_of_week'].apply(lambda x: 1 if x in [5, 6] else 0)
            df['is_rush_hour'] = df['Pickup_hour'].apply(lambda x: 1 if x in [7, 8, 9, 16, 17,18,19] else 0)
            df['is_night_hour'] = df['Pickup_hour'].apply(lambda x: 1 if x in [0,1,2,3,4,5,6,20,21,22,23] else 0)

            ## creating the season of the year
            df['season_of_year'] = df['pickup_month'].apply(lambda x: 'winter' if x in [12, 1, 2]
                                                            else 'spring' if x in [3,4,5] else
                                                            'summer' if x in [6,7,8] else 'autumn')


            ## Deriving the Holdiay feature
            def is_holiday(data:pd.DataFrame):
                # Weekends
                if data['day_of_week'] in [5, 6]:  # Saturday=5, Sunday=6
                    return 1

                # Fixed-date holidays
                fixed_holidays = [(1, 1), (7, 4), (11, 11), (6, 19), (12, 25)]
                if (data['pickup_month'], data['day_of_month']) in fixed_holidays:
                    return 1

                # Movable holidays
                date = pd.Timestamp(year=data['pickup_year'], month=data['pickup_month'], day=data['day_of_month'])

                # Last Thursday in November
                if data['pickup_month'] == 11 and date.weekday() == 3 and date + pd.Timedelta(
                    days=7) > pd.Timestamp(year=data['pickup_year'], month=11, day=30):
                    return 1

                # Last Monday in May
                if data['pickup_month'] == 5 and date.weekday() == 0 and date + pd.Timedelta(days=7) > pd.Timestamp(
                    year=data['pickup_year'], month=5, day=31):
                    return 1

                # Third Monday in January
                if data['pickup_month'] == 1 and date.weekday() == 0 and 15 <= data['day_of_month'] <= 21:
                    return 1

                # First Monday in September
                if data['pickup_month'] == 9 and date.weekday() == 0 and 1 <= data['day_of_month'] <= 7:
                    return 1

                # First Tuesday in November
                if data['pickup_month'] == 11 and date.weekday() == 1 and 1 <= data['day_of_month'] <= 7:
                    return 1

                # Second Monday in October
                if data['pickup_month'] == 10 and date.weekday() == 0 and 8 <= data['day_of_month'] <= 14:
                    return 1

                return 0
            
            df['is_holiday'] = df.apply(is_holiday, axis=1)

            ## creating a column for special event
            def is_special_event(data):
                # Fixed-date holidays
                fixed_holidays = [(3, 17), (7, 4), (6, 4), (12,31), (6, 5), (6, 6), (6,7),(6,8),
                                (6,9),(6,10), (6,11), (6,12), (6,13),(6,14),(6,15)]
                if (data['pickup_month'], data['day_of_month']) in fixed_holidays:
                    return 1

                # Movable holidays
                date = pd.Timestamp(year=data['pickup_year'], month=data['pickup_month'], day=data['day_of_month'])

                # Macy's Thanksgiving Parade
                if data['pickup_month'] == 11 and date.weekday() == 3 and date + pd.Timedelta(
                    days=7) > pd.Timestamp(year=data['pickup_year'], month=11, day=30):
                    return 1

                # Last Sunday in June (Pride Month)
                if data['pickup_month'] == 6 and date.weekday() == 6 and date + pd.Timedelta(days=7) > pd.Timestamp(
                    year=data['pickup_year'], month=6, day=30):
                    return 1

                return 0

            df['Is_special_event'] = df.apply(is_special_event, axis=1) ## creating the feature

            ## creating a column for Payday Indicator
            def is_payday(data):

                date = pd.Timestamp(year=data['pickup_year'], month=data['pickup_month'], day=data['day_of_month'])

                if date.is_month_start or data['day_of_month']==15 or date.is_month_end:
                    return 1

                return 0

            df['is_payday'] = df.apply(is_payday, axis=1) ##deriving the payday indicator

            return df
        
        except Exception as e:
            raise RideDemandException(e,sys)
        

    def engineer_autoregressive_signals(self, df:pd.DataFrame)->pd.DataFrame:

        try:
            def make_lags(group, col='pickups'):

                ## for lag features
                for l in [1,24]:
                    group[f'{col}_lag_{l}'] = group[col].shift(l)

                ## for rolling mean and std
                for w in [24]:
                    group[f'{col}_roll_mean_{w}'] = group[col].shift(1).rolling(w).mean()
                    group[f'{col}_roll_std_{w}'] = group[col].shift(1).rolling(w).std()
                return group
            
            df.reset_index()
            df = df.sort_values(['PULocationID','bin'])
            df = df.groupby('PULocationID', group_keys=False).apply(make_lags) ##generating the autoregressive feature

            return df

        except Exception as e:
            logger.error('Failed to generate the features', e)
            raise RideDemandException(e,sys)
        
    def city_wide_congestion_features(self, df:pd.DataFrame)-> pd.DataFrame:
        try:
            taxi_df = self.taxi_df

            ## computing the trip distance
            taxi_df['trip_duration_hr'] = (
                (taxi_df['tpep_dropoff_datetime'] - taxi_df['tpep_pickup_datetime'])
                .dt.total_seconds() / 3600
            )

            ## computing the MPH
            taxi_df['MPH']= taxi_df['trip_distance']/taxi_df['trip_duration_hr']

            ## filtering out invalid mph
            ## based on research, traffic in new york city rarely goes above 60mph
            taxi_df = taxi_df[
                taxi_df['MPH'].between(1, 60)  # min 1 mph, max 60 mph
            ]

            ## creating the city-wide congestion
            city_speed = (taxi_df
                .groupby('bin')['MPH']
                .mean()
                .rename('city_avg_speed')
                .reset_index()
            )

            ## computing the congestion index by inverting the city_avg_speed
            city_speed['city_congestion_index'] = 1 / city_speed['city_avg_speed']

            # Merge City speed into the main table
            df = df.merge(city_speed, on=['bin'], how='left')

            return df

        except Exception as e:
            logger.error("Unable to generate the city wide features",e)
            raise RideDemandException(e,sys)
        

    def zone_level_congestion_features(self, df:pd.DataFrame)->pd.DataFrame:
        try:
            taxi_df = self.taxi_df

            ## computing the zone level congestion
            zone_speed = (taxi_df
                .groupby(['PULocationID', 'bin'])['MPH']
                .mean()
                .rename('zone_avg_speed')
                .reset_index()
            )

            # Inverse speed as congestion index
            zone_speed['zone_congestion_index'] = 1 / zone_speed['zone_avg_speed']

            # Build full grid to include zeros
            zones = df['PULocationID'].unique()
            time_index = pd.date_range(df['bin'].min(), df['bin'].max(), freq='60min')
            grid = pd.MultiIndex.from_product([zones, time_index], names=['PULocationID','bin']).to_frame(index=False)

            zone_speed = grid.merge(zone_speed, how='left', on=['PULocationID','bin']).fillna({'zone_avg_speed':0, 'zone_congestion_index':0})

            # Merge the Zone_speed into the main table
            df = df.merge(zone_speed, on=['PULocationID','bin'], how='left')

            return df

        except Exception as e:
            logger.error('Unable to generate the zone level features', e)
            raise RideDemandException(e,sys)
        
    def citywide_hourly_demand(self, df:pd.DataFrame)->pd.DataFrame:
        try:
            # Citywide hourly demand
            city_demand = (df.groupby('bin')['pickups']
                            .sum()
                            .rename('city_pickups')
                            .reset_index())

            # Merge into zone-hour table
            df = df.merge(city_demand, on='bin', how='left')

            # Create lag features (1h, 24h)
            for lag in [1, 24]:
                df[f'city_pickups_lag_{lag}h'] = df['city_pickups'].shift(lag)

            return df

        except Exception as e:
            logger.error("Unable to engineer the citywide hourly demand features", e)
            raise RideDemandException(e,sys)
        
    def generate_neighbor_features(self, df:pd.DataFrame)->pd.DataFrame:
        try:
            zones_gdf = load_shapefile_from_zip(self.config.taxi_zone_shapefile_url,
                                               self.config.shapefile_dir)
            logger.info(f'loaded the shapefile successfully to {self.config.shapefile_dir}')

            # Spatial join: each zone with all zones it touches
            neighbors_df = gdp.sjoin(zones_gdf, zones_gdf, how="left", predicate="touches")

            # Remove self-joins
            neighbors_df = neighbors_df[neighbors_df['LocationID_left'] != neighbors_df['LocationID_right']]

            # Group into a dictionary: zone -> list of neighbor zones
            neighbor_dict = (neighbors_df.groupby('LocationID_left')['LocationID_right']
                            .apply(list)
                            .to_dict())
            
            # df: your zone-hour pickup table
            # Columns: ['PULocationID', 'bin', 'pickups']
            # neighbor_dict: {zone_id: [neighbor_zone_ids, ...]}

            # Step 1: Create a DataFrame mapping each zone to its neighbors
            neighbor_pairs = []
            for zone, neighs in neighbor_dict.items():
                for n in neighs:
                    neighbor_pairs.append((zone, n))

            neighbor_df = pd.DataFrame(neighbor_pairs, columns=['PULocationID', 'neighbor_id'])

            # Step 2: Join neighbor_df with df to get neighbor pickups
            # Rename df columns for clarity before merge
            df_neighbors = df.rename(columns={'PULocationID': 'neighbor_id', 'pickups': 'neighbor_pickups'})
            df_neighbors.reset_index(inplace=True)

            # Merge: for each (zone, neighbor), bring in neighbor's pickups for each hour
            merged = neighbor_df.merge(df_neighbors, on='neighbor_id', how='left')


            # Step 3: Group by zone and hour to sum neighbor pickups
            neighbor_demand_df = (merged
                .groupby(['PULocationID', 'bin'])['neighbor_pickups']
                .sum()
                .reset_index()
                .rename(columns={'neighbor_pickups': 'neighbor_pickups_sum'})
            )

            # Step 4: Merge back into your main df
            df = df.reset_index()
            df = df.merge(neighbor_demand_df, on=['PULocationID', 'bin'], how='left')
            df['neighbor_pickups_sum'] = df['neighbor_pickups_sum'].fillna(0)

            ## computing the Lagged neighbor demand
            for lag in [1, 24]:
                df[f'neighbor_pickups_lag_{lag}h'] = df.groupby('PULocationID')['neighbor_pickups_sum'].shift(lag)

            # Function to sum neighbor demand
            #def neighbor_demand(row):
             #   neighbors = neighbor_dict
              #  neigh_ids = neighbors.get(row['PULocationID'], [])
               # mask = (df['PULocationID'].isin(neigh_ids)) & (df['bin'] == row['bin'])
                #return df.loc[mask, 'pickups'].sum()

            ## handling the missing data caused by lag, rolling mean/std
            df = df.fillna(method='bfill')

        except Exception as e:
            logger.error("Unable to generate the Neighbor features", e)
            raise RideDemandException(e,sys)
        
    def save_data_to_feature_store(self):
        try:
            df = self.generate_neighbor_features()
            transformed_data_store = self.config.transformed_data_file_path
            logger.info(f"Saving the transformed dataset to the feature store")

            df.to_csv(transformed_data_store, index=False)
            logger.info(f'Transformed data saved to path, {transformed_data_store}')

            print(f"size of Transformed data: {df.shape}")

        except Exception as e:
            logger.error("Unable to save the file",e)
            raise RideDemandException(e,sys)
    
    #def feature_engineering(self):




In [5]:
from dateutil.relativedelta import relativedelta
from datetime import timedelta, datetime

In [8]:
now = (datetime.today())#- relativedelta(months=1))
# end_date = datetime.strptime(now, "%Y-%m-%d") ## converting to datetime
end_date = now - timedelta(days=now.day) ## retrieving the last day of the previous month

## accessing the previous month
days_to_subtract = time_subtract(end_date.strftime('%Y-%m-%d'))
end_date = (end_date - timedelta(days=days_to_subtract))

## start of the month
days= time_subtract(end_date.strftime('%Y-%m-%d'))
start_date= end_date - timedelta(days=days-1)

start_date = start_date.strftime('%Y-%m-%d')
end_date = end_date.strftime('%Y-%m-%d')

print(f'start date: {start_date}, end date: {end_date}')

start date: 2025-11-01, end date: 2025-11-30


In [ ]:
#from src.DynamicPricingEngine.utils.data_transformation_utils import DataPusher

In [ ]:
data = ""#DataPusher()
df = data._reliable_insert()

In [16]:
import requests

In [ ]:
from pydantic import BaseModel, field_validator
import pandas as pd
import logging

# 1. Improved Pydantic Model
class PredictionRequest(BaseModel):
    datetime_val: str # Renamed from 'datetime' to avoid confusion with module
    pulocationid: int

    @field_validator('datetime_val', mode='before')
    @classmethod
    def parse_datetime(cls, v):
        return pd.to_datetime(v)

    @field_validator('pulocationid')
    @classmethod
    def check_pulocationid(cls, v):
        if not (1 <= v <= 263):
            raise ValueError("PULocationID must be between 1-263")
        return v

class InferencePipeline:
    # ... (init logic is mostly fine, but ensure you pass the Hopsworks FS object) ...
    def __init__(self, config, start_date_time: str, 
                 end_date_time:str, api_key: str, pulocationid: int):
        try:
            self.config = config
            if not isinstance(start_date_time, str):
                raise ValueError("start_date_time must be a string in 'YYYY-MM-DD' format")
            self.date = datetime.now()

            if not isinstance(end_date_time, str):
                raise ValueError("end_date_time must be a string in 'YYYY-MM-DD' format")
            self.end_date = end_date_time

            self.api_key = api_key

            if not (1<=pulocationid<=263):
                raise ValueError("PULocationID must be between 1-263")
            self.pulocationid = pulocationid

        except Exception as e:
            logger.error("Error initializing Inference Pipeline", e)
            raise RideDemandException(e, sys)

    def get_nyc_prediction_weather_data(self)-> pd.DataFrame:

        try:
            base_url = self.config.weather_data_url
            location: str = "New York, NY, United States"
            start_of_month_str = self.start_date
            end_of_month_str = self.end_date
            api_key = self.api_key

            params = {
                "unitGroup": "us",
                "key": api_key,
                "include": "days,hours,current,alerts,stations",
                "contentType": "json"
            }

            url = f"{base_url}/{location}/{start_of_month_str}/{end_of_month_str}"
            logger.info(f"Fetching data from {start_of_month_str} to {end_of_month_str}...")

            try:
                response = requests.get(url, params=params)
                response.raise_for_status()

            except requests.RequestException as e:
                print(f"Request failed: {e}")
                return None

            data = response.json()
            days = data.get("days", [])
            if not days:
                print("No 'days' data found in response.")
                return None

            hourly_records = []
            fields = ['datetime', 'temp', 'humidity']

            for day in days:
                for hour in day.get("hours", []):
                    filtered_hour = {key: hour.get(key) for key in fields}
                    filtered_hour["day"] = day.get("datetime")
                    hourly_records.append(filtered_hour)

            df_hours = pd.DataFrame(hourly_records)
            logger.info(f"Retrieved {len(days)} days ({df_hours.shape[0]} hourly records).")

            return df_hours

        except Exception as e:
            logger.error(f"failed to extract weather data {e}")
            raise RideDemandException(e,sys)

    def engineer_temporal_prediction_features(self, df: pd.DataFrame)-> pd.DataFrame:
        try:
            pulocationid = self.pulocationid
            df['datetime']= pd.to_datetime(df['day'] + ' ' + df['datetime'])
            df.drop(columns=['day'], inplace=True)

            # Extract temporal features
            df['pickup_hour'] = df['datetime'].dt.hour
            df['is_rush_hour'] = df['hour'].apply(lambda x: 1 if 7 <= x <= 9 or 16 <= x <= 19 else 0)
            df['pulocationid'] = pulocationid

            logger.info("Temporal Feature for prediction generated successfully.")
            return df

        except Exception as e:
            logger.error(f"failed to engineer temporal features for prediction data {e}")
            raise RideDemandException(e,sys)

    def generate_lag_features_for_prediction(self, weather_pred_df: pd.DataFrame) -> pd.DataFrame:
        """
        FIXED: You must fetch historical demand from Hopsworks to create lags.
        """
        try:
            # 1. Connect to Hopsworks Feature Store (Assumes self.fs is initialized)
            # You need historical demand for the specific location
            fg = self.fs.get_feature_group(name="taxi_demand_feature_group", version=1)
            
            # 2. Fetch last 24-48 hours of ACTUAL demand to create lags for the NEW prediction
            # Use hopsworks query or read_feature_group
            hist_df = fg.filter(fg.pulocationid == self.pulocationid).read()
            hist_df = hist_df.sort_values('datetime')

            # 3. Combine with your weather_pred_df to ensure continuous timeline
            full_df = pd.concat([hist_df, weather_pred_df]).sort_values('datetime')

            # 4. Generate actual lags on the 'target' column (e.g., 'ride_count')
            full_df['pickup_lag1'] = full_df['ride_count'].shift(1)
            full_df['pickup_lag24'] = full_df['ride_count'].shift(24)
            
            full_df['pickup_roll_mean24'] = full_df['ride_count'].shift(1).rolling(window=24).mean()

            # 5. Return only the rows that belong to your prediction window
            return full_df[full_df['datetime'] >= weather_pred_df['datetime'].min()]

        except Exception as e:
            logger.error(f"Failed to generate lag features: {e}")
            raise

In [12]:
import requests

In [13]:
def get_nyc_prediction_weather_data(weather_data_url):

    try:
        #base_url = weather_data_url
        location: str = "New York, NY, United States"
        base_url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{location}/today"
        #api_key = self.api_key

        params = {
            "unitGroup": "us",
            "key": 'HT2ZYDZG8J25XYFP2E9ABUXJF',
            "include": "hours,current,alerts",
            "contentType": "json"
        }

        url = f"{base_url}/{location}/"
        logger.info(f"Fetching weather data ...")

        try:
            response = requests.get(url, params=params)
            response.raise_for_status()

        except requests.RequestException as e:
            print(f"Request failed: {e}")
            return None

        data = response.json()
        #days = data.get("days", [])
        #if not days:
         #   print("No 'days' data found in response.")
          #  return None
        
        # Access specific data (e.g., current conditions)
        current = data.get("currentConditions")
        print(f"Current temp in NYC: {current.get('temp')}°F")

        hourly_records = []
        #fields = ['datetime', 'temp', 'humidity']

        #for day in days:
         #   for hour in day.get("hours", []):
          #      filtered_hour = {key: hour.get(key) for key in fields}
           #     filtered_hour["day"] = day.get("datetime")
        hourly_records.append(current)

        df_hours = pd.DataFrame(hourly_records)
        #logger.info(f"Retrieved {len(days)} days ({df_hours.shape[0]} hourly records).")

        return df_hours

    except Exception as e:
        logger.error(f"failed to extract weather data {e}")
        raise RideDemandException(e,sys)

In [ ]:
import requests
def nyc_current_weather_data():
    # Define your API configuration
    API_KEY = ""  # Replace with your actual key
    LOCATION = "New York, NY, United States"
    BASE_URL = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline"
    
    
    # Construct the full URL
    url = f"{BASE_URL}/{LOCATION}/today"

    # Set up the query parameters
    params = {
        "unitGroup": "us",
        "include": "current",
        "key": API_KEY,
        "contentType": "json"
    }

    try:
        # Execute the GET request
        response = requests.get(url, params=params)
        
        # Check if the request was successful
        response.raise_for_status()
        
        # Parse the JSON data
        data = response.json()
        
        # Access specific data (e.g., current conditions)
        current = data.get("currentConditions")
        hourly_records = []
        hourly_records.append(current)

        df_hours = pd.DataFrame(hourly_records)
        df_hours = df_hours[['datetime', 'temp', 'humidity']]
        df_hours['datetime'] = pd.to_datetime(df_hours['datetime'])
        
        return df_hours
        
    except requests.exceptions.HTTPError as err:
        print(f"HTTP error occurred: {err}")
    except Exception as err:
        print(f"An error occurred: {err}")

In [18]:
import pandas as pd

In [19]:
from src.DynamicPricingEngine.logger.logger import logger
from src.DynamicPricingEngine.exception.customexception import RideDemandException
import os, sys
import requests


In [8]:
from hsfs.feature import Feature

In [ ]:
api_key = ""
#config= config
cat_cols = ['pickup_hour','is_rush_hour']

def retrieve_engineered_feature():
    try:
        now = datetime.today()
        end_date = now - timedelta(days=now.day) ## retrieving the last day of the previous month

        ## accessing the previous month
        days_to_subtract = time_subtract(end_date.strftime('%Y-%m-%d'))
        end_date = (end_date- timedelta(days=days_to_subtract)+ timedelta(days=1))

        ## a year back from end date 
        start_date = end_date - relativedelta(months= 1)

        start_date = start_date.strftime('%Y-%m-%d')
        end_date = (end_date-relativedelta(days=1)).strftime('%Y-%m-%d')

        logger.info('Retrieving the dataset from hopsworks feature store')

        ## login to feature store
        #project = hopsworks.login(project='RideDemandPrediction', api_key_value=api_key)
        project = hopsworks.login(api_key_value=api_key)
        fs = project.get_feature_store()

        # Get the feature group
        fg = fs.get_feature_group(name="ridedemandprediction", version=1)

        features = [
            'bin',
            'temp',
            'humidity',
            'pickup_hour',
            'is_rush_hour',
            'city_avg_speed',
            'zone_avg_speed',
            'zone_congestion_index',
            'pickups_lag_1h',
            'pulocationid',
            'pickups_lag_24h',
            'city_pickups_lag_1h',
            'neighbor_pickups_lag_1h',
            'pickups',
            'datetime'
        ]

        query=fg.select(features).filter((Feature('bin_str') >= start_date) & (Feature('bin_str') <= end_date))

        # creating a feature view
        df = query.read()
        logger.info('Feature data retrieved successfully from the feature store')
    
        df.set_index(['bin'], inplace=True)

        return df
    
    except Exception as e:
        logger.error(f"Error retrieving the dataset, {e}")
        raise RideDemandException(e,sys)

In [65]:
from hsfs.feature_view import FeatureView

In [74]:
def retrieve_engineered_feature():
        try:
            ## login to feature store
            
            now = datetime.today()
            end_date = now - timedelta(days=now.day) ## retrieving the last day of the previous month

            ## accessing the previous month
            days_to_subtract = time_subtract(end_date.strftime('%Y-%m-%d'))
            end_date = (end_date- timedelta(days=days_to_subtract)+ timedelta(days=1))

            ## a year back from end date 
            start_date = end_date - relativedelta(months= 1)

            start_date = start_date.strftime('%Y-%m-%d')
            end_date = end_date.strftime('%Y-%m-%d')

            logger.info('Retrieving the dataset from hopsworks feature store')


            ## login to feature store
            project = hopsworks.login(project='RideDemandPrediction', api_key_value=api_key)
            fs = project.get_feature_store()

            # Get the feature group
            fg = fs.get_feature_group(name="ridedemandprediction", version=1)
            query=fg.select([
                'bin',
                'temp',
                'humidity',
                'pickup_hour',
                'is_rush_hour',
                'city_avg_speed',
                'zone_avg_speed',
                'zone_congestion_index',
                'pickups_lag_1h',
                'pulocationid',
                'pickups_lag_24h',
                'city_pickups_lag_1h',
                'neighbor_pickups_lag_1h',
                'pickups'
            ])

            # delete the previous month feature view data
            FeatureView.clean(feature_store_id=fs._id, 
                                feature_view_name='ride_demand_fv',
                                feature_view_version=1)

            # create a new feature view from the feature group
            feature_view = fs.create_feature_view(name="ride_demand_fv",
                                                    version=1,
                                                    description="Features for ride demand prediction",
                                                    query=query)

            logger.info('hopsworks feature view created successfully')

            # Materialize training dataset using Spark job
            version, jobs = feature_view.create_training_data(start_time = start_date,
                                                                end_time = end_date,
                                                                description="365 days ride demand training data",
                                                                data_format="parquet",
                                                                write_options = {'use_spark': True}
                                                                )

            logger.info('Training data created successfully and materialized in hopsworks')
            logger.info(f"Data from {start_date} to {end_date} created and materialized Successfully")

            df, _ = feature_view.get_training_data(training_dataset_version=1,
                                                read_options={"use_hive":False})
            
            logger.info('Data successfully retrieved from the feature store')
            
            df.set_index(['bin'], inplace=True)

            return df
        
        except Exception as e:
            logger.error(f"Error retrieving the dataset, {e}")
            raise RideDemandException(e,sys)

In [11]:
def retrieve_engineered_feature(self):
        try:
            ## login to feature store
            now = datetime.today()
            end_date = now - timedelta(days=now.day) ## retrieving the last day of the previous month

            ## accessing the previous month
            days_to_subtract = time_subtract(end_date.strftime('%Y-%m-%d'))
            end_date = (end_date- timedelta(days=days_to_subtract)+ timedelta(days=1))

            ## a year back from end date 
            start_date = end_date - relativedelta(months= 1)

            start_date = start_date.strftime('%Y-%m-%d')
            end_date = end_date.strftime('%Y-%m-%d')

            logger.info('Retrieving the dataset from hopsworks feature store')


            ## login to feature store
            project = hopsworks.login(project='RideDemandPrediction', api_key_value=self.api_key,
                                  #    config={'connect_timeout': 300, 'read_timeout': 300} 
                                        )
            fs = project.get_feature_store()

            # Get the feature group
            #fg = fs.get_feature_group(name="ridedemandprediction", version=1)
            #query=fg.select_all()
            
            feature_view = fs.get_feature_view(name="ride_demand_fv",
                                                    version=1)

            #logger.info('hopsworks feature view created successfully')

            try:
                feature_view.delete_training_dataset(training_dataset_version=1)
                logger.info("Training data version 1 deleted.")

                # Materialize training dataset using Spark job
                version, jobs = feature_view.create_training_data(start_time = start_date,
                                                                    end_time = end_date,
                                                                    description="365 days ride demand training data",
                                                                    data_format="parquet",
                                                                    write_options = {'use_spark': True}
                                                                    )

                logger.info('Training data created successfully and materialized in hopsworks')
                logger.info(f"Data from {start_date} to {end_date} created and materialized Successfully")


            except:
                # Materialize training dataset using Spark job
                version, jobs = feature_view.create_training_data(start_time = start_date,
                                                                    end_time = end_date,
                                                                    description="365 days ride demand training data",
                                                                    data_format="parquet",
                                                                    write_options = {'use_spark': True}
                                                                    )

                logger.info('Training data created successfully and materialized in hopsworks')
                logger.info(f"Data from {start_date} to {end_date} created and materialized Successfully")

            df, _ = feature_view.get_training_data(training_dataset_version=1,
                                                read_options={"use_hive":False})
            
            logger.info('Data successfully retrieved from the feature store')
            
            df.set_index(['bin'], inplace=True)

            return df
        
        except Exception as e:
            logger.error(f"Error retrieving the dataset, {e}")
            raise RideDemandException(e,sys)

In [ ]:
def extract_historical_pickup_data()-> pd.DataFrame:
        try:
            cat_cols = ['pickup_hour','is_rush_hour']

            # Define the time range for historical data extraction
            now = datetime.now()
            end_date = datetime.now() - timedelta(minutes=now.minute, 
                                                  seconds=now.second, 
                                                  microseconds=now.microsecond)
            start_date = end_date - timedelta(hours=24)   

            logger.info('Retrieving the dataset from hopsworks feature store')

            ## login to feature store
            project = hopsworks.login(project='RideDemandPrediction', api_key_value=api_key,
                                    )
            fs = project.get_feature_store()

            # Get the feature group
            fg = fs.get_feature_group(name="ridedemandprediction", version=1)

            final_features = [
                'temp',
                'humidity',
                'pickup_hour',
                'is_rush_hour',
                'city_avg_speed',
                'zone_avg_speed',
                'zone_congestion_index',
                'pickups_lag_1h',
                'pulocationid',
                'pickups_lag_24h',
                'city_pickups_lag_1h',
                'neighbor_pickups_lag_1h',
                'pickups'
            ]
            query=fg.select(final_features)

                ## login to feature store
            project = hopsworks.login(project='RideDemandPrediction', api_key_value=api_key)
            fs = project.get_feature_store()

            # Get the feature group
            #fg = fs.get_feature_group(name="ridedemandprediction", version=1)
            #query=fg.select_all()
            
            feature_view = fs.create_feature_view(name="ride_demand_fv",
                                                    version=1)

            #logger.info('hopsworks feature view created successfully')

            # creating a feature view
            # create a new feature view from the feature group
            #feature_view = fs.create_feature_view(name="ride_demand_prediction_fv",
                                                   # version=1,
                                                  #  description="ride demand historical data for prediction",
                                                 #   query=query)

            logger.info('hopsworks feature view created successfully')

            feature_view = fs.get_feature_view(name='ride_demand_prediction_fv', version= 1)

            # Materialize training dataset using Spark job
            version, jobs = feature_view.create_training_data(start_time = start_date,
                                                                end_time = end_date,
                                                                description="ride demand training data",
                                                                data_format="parquet",
                                                                write_options = {'use_spark': True}
                                                                )

            logger.info('Training data created successfully and materialized in hopsworks')
            logger.info(f"Data from {start_date} to {end_date} created and materialized Successfully")

            feature_view = fs.get_feature_view(name='ride_demand_fv', version= 1)

            df, _ = feature_view.get_training_data(training_dataset_version=1,
                                                read_options={"use_hive":False})
            
            logger.info('Data successfully retrieved from the feature store')
            
            df.set_index(['bin'], inplace=True)

            return df
        
        except Exception as e:
            logger.error(f"failed to extract historical pickup data {e}")
            raise RideDemandException(e, sys)


In [1]:
import pandas as pd
import hopsworks
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import sys
from hsfs.feature import Feature

/home/emmanuel/Desktop/Dynamic-Pricing-Engine/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def leap_year(year:int)->bool:

    try:

        if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
            return True
        else:
            return False

    except Exception as e:
        raise e


def time_subtract(date:str)->int:
    """
    A function to calculate the number of days to subract from the data ingesting
    Args:
        date (str): the date to use
    Return:
        days(int): the number of days calculated
       """
    try:

        date = datetime.strptime(date, "%Y-%m-%d")
        days = 0

        if (date.month != 2) and (date.month not in (4,9,6,11)):
            days=31

        elif (date.month != 2) and (date.month in (4,9,6,11)):
            days=30

        elif (date.month == 2) and (leap_year(date.year)):
            days=29

        else:
            days=28

        return days

    except Exception as e:
        raise e

In [ ]:
api_key = ""

In [4]:
from datetime import datetime
from zoneinfo import ZoneInfo
from dateutil.relativedelta import relativedelta

def extract_historical_pickup_data()-> pd.DataFrame:
    try:
      ## login to feature store
      project = hopsworks.login(project='RideDemandPrediction', api_key_value=api_key)
      fs = project.get_feature_store()

      final_features = [
              'temp', 'humidity', 'pickup_hour', 'is_rush_hour', 'city_avg_speed', 'zone_avg_speed',
              'zone_congestion_index', 'pickups_lag_1h', 'pulocationid', 'pickups_lag_24h', 'city_pickups_lag_1h',
              'neighbor_pickups_lag_1h', 'neighbor_pickups_sum', 'pickups', 'bin_str', 'bin']

      ## creating a new feature group
      prediction_fg = fs.create_feature_group(
          name="demand_predictions",
          version=1,
          primary_key=['pulocationid', 'bin_str'],
          event_time='bin',
          online_enabled=False,
          description="Logs of model predictions for evaluation"
        )
      print('feature group created successfully')

      # Defining the time range for historical data extraction
      now = datetime.now().replace(hour=0,minute=0, second=0, microsecond=0)
      end_date = now - timedelta(days=now.day) ## retrieving the last day of the previous month

      ## accessing the previous month
      days_to_subtract = time_subtract(end_date.strftime('%Y-%m-%d'))
      end_date = (end_date- timedelta(days=days_to_subtract)+ timedelta(days=1))

      ## a year back from end date
      start_date = end_date - relativedelta(days= 1)

      start_date = start_date.strftime('%Y-%m-%d %H:%M:%S')
      end_date = end_date.strftime('%Y-%m-%d %H:%M:%S')
      print(start_date, end_date)

      print('Retrieving the dataset from hopsworks feature store')

      # Get the feature group
      train_fg = fs.get_feature_group(name="ridedemandprediction", version=1)
      query=train_fg.select(final_features).filter(train_fg.bin >= start_date).filter(train_fg.bin <= end_date)
      historic_df = query.read()

      print('data successfully retrieved from the feature store')

      prediction_fg.insert(historic_df, storage = 'offline', write_options = {'wait_for_job': True, 'use_spark':True})
      print('data successfuly inserted')

      return historic_df

    except Exception as e:
      print(f"failed to extract historical pickup data {e}")
      raise e

In [7]:
# Defining the time range for historical data extraction
now = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
end_date = now - timedelta(days=now.day) ## retrieving the last day of the previous month

## accessing the previous month
days_to_subtract = time_subtract(end_date.strftime('%Y-%m-%d'))
end_date = (end_date- timedelta(days=days_to_subtract)+ timedelta(days=1))

## a year back from end date
start_date = end_date - relativedelta(days= 1)

start_date = start_date.strftime('%Y-%m-%d %H:%M:%S')
end_date = end_date.strftime('%Y-%m-%d %H:%M:%S')
print(start_date, end_date)


2025-11-30 00:00:00 2025-12-01 00:00:00


In [105]:
from src.DynamicPricingEngine.logger.logger import logger
from src.DynamicPricingEngine.exception.customexception import RideDemandException
import hopsworks
import os, sys
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [ ]:
def get_nyc_prediction_weather_data()-> pd.DataFrame:

    try:
        # Define your API configuration
        ny_tz = ZoneInfo("America/New_York")
        api_key = ""
        location = "New York, NY, United States"
        base_url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline"
    
        params = {
            "unitGroup": "us",
            "key": api_key,
            "include": "current",
            "contentType": "json"
        }

        url = f"{base_url}/{location}/today"
        logger.info(f"Fetching data for current timestamp {datetime.now(ny_tz).strftime('%Y-%m-%d, %H')}:00:00 ...")

        try:
            response = requests.get(url, params=params)
            response.raise_for_status()

        except requests.RequestException as e:
            print(f"Request failed: {e}")
            return None

        # Access specific data (e.g., current conditions)
        response = response.json()
        current = response.get("currentConditions")
        hourly_records = []
        hourly_records.append(current)

        df_hours = pd.DataFrame(hourly_records)
        df_hours = df_hours[['datetime', 'temp', 'humidity']]
        df_hours['datetime'] = pd.to_datetime(df_hours['datetime']).dt.floor('H')+ timedelta(hours=1)

        logger.info(f"Successfully Retrieved the weather data for {datetime.now(ny_tz).strftime('%Y-%m-%d, %H')}:00:00.")

        return df_hours

    except Exception as e:
        logger.error(f"failed to extract weather data {e}")
        raise RideDemandException(e,sys)

In [107]:
def engineer_temporal_prediction_features(weather_df: pd.DataFrame)-> pd.DataFrame:
    try:
        #pulocationid = self.pulocationid
        # Extract temporal features
        weather_df['pickup_hour'] = weather_df['datetime'].dt.hour
        #weather_df['day_of_week'] = weather_df['datetime'].dt.dayofweek
        weather_df['is_rush_hour'] = weather_df['pickup_hour'].apply(lambda x: 1 if 7 <= x <= 9 or 16 <= x <= 19 else 0)
      #  weather_df['pulocationid'] = pulocationid
        weather_df.rename(columns={'datetime':'bin'}, inplace=True)

        
        logger.info("Temporal Feature for prediction generated successfully.")
        return weather_df

    except Exception as e:
        logger.error(f"failed to engineer temporal features for prediction data {e}")
        raise RideDemandException(e,sys)


In [108]:
def extract_historical_pickup_data(start_date=None, end_date=None) -> pd.DataFrame:
    try:
        project = hopsworks.login(project='RideDemandPrediction', api_key_value=api_key)
        fs = project.get_feature_store()

        ny_tz = ZoneInfo("America/New_York")
        now_ny = datetime.now(ny_tz).replace(minute=0, second=0, microsecond=0)

        start_date_utc = (now_ny - relativedelta(hours=24))
        end_date_utc = now_ny

        fh_start = start_date_utc.strftime('%Y-%m-%d %H:%M:%S')
        fh_end = end_date_utc.strftime('%Y-%m-%d %H:%M:%S')

        prediction_fg = fs.get_or_create_feature_group(
            name="demand_predictions",
            version=1,
            primary_key=['pulocationid', 'bin_str'],
            event_time='bin',
            description="Logs of model predictions for evaluation"
        )
        if start_date is None or end_date is None:
          start_date =fh_start
          end_date = fh_end

        final_features = [
              'temp', 'humidity', 'pickup_hour', 'is_rush_hour', 'city_avg_speed', 'zone_avg_speed',
              'zone_congestion_index', 'pickups_lag_1h', 'pulocationid', 'pickups_lag_24h', 'city_pickups_lag_1h',
              'neighbor_pickups_lag_1h', 'neighbor_pickups_sum', 'pickups', 'bin_str', 'bin']

        # Use the explicit query format
        query = prediction_fg.select(final_features).filter(
            (prediction_fg.bin >= start_date) & (prediction_fg.bin <= end_date)
        )

        historic_df = query.read()

        print(f"Successfully retrieved {len(historic_df)} rows for window: {start_date} to {end_date}")

        return historic_df

    except Exception as e:
        print(f"Failed to extract historical pickup data: {e}")
        raise e

In [ ]:
def generate_lag_features_for_prediction(self, hist_df:pd.DataFrame,
                                              pred_df: pd.DataFrame)-> pd.DataFrame:
        try:
            #merging historical and prediction data
            df = pd.concat([hist_df, pred_df], axis = 0, ignore_index=True)
            df = df.sort_values(by='datetime')
            df.set_index('datetime', inplace=True)

            # getting only the last 24 hours of historical data
            df = df.last('24H')

            ## creating citywide pickup counts
            df_city = df.groupby('datetime').agg({'pickups':'sum'}).rename(columns={'pickups':'city_pickups'})
            df = df.merge(df_city, on='datetime', how='left')

            # city avg speed
            df['city_avg_speed'] = df['city_pickups'] / df['city_trip_distance']

            ## creating neighbor pickup counts
            df_neigh = df.groupby('pulocationid').agg({'pickups':'sum'}).rename(columns={'pickups':'neighbor_pickups_sum'})
            df = df.merge(df_neigh, on='pulocationid', how='left')

            ##  Final features needed for prediction
               # 'city_avg_speed',
              #  'zone_avg_speed',
             #   'zone_congestion_index',
            #    'pickups_lag_1h',
           #     'pulocationid',
          #      'pickups_lag_24h',
         #       'city_pickups_lag_1h',
           #     'neighbor_pickups_lag_1h'



            # Generating lag features
            df['pickup_lag1'] = df['pickups'].shift(1)
            df['pickup_lag24'] = df['pickups'].shift(24)
            
            df.reset_index(inplace=True)

            logger.info("Lag features for prediction generated successfully.")
            return df

        except Exception as e:
            logger.error(f"failed to engineer lag features for prediction data {e}")
            raise RideDemandException(e,sys)

In [ ]:
import pandas as pd
import requests
from datetime import datetime

def get_nyc_prediction_weather_data() -> pd.DataFrame:
    try:
        api_key = "" 
        location = "New York, NY, United States"
        base_url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline"

        params = {
            "unitGroup": "us",
            "key": api_key,
            "include": "current",
            "contentType": "json"
        }

        url = f"{base_url}/{location}/today"
        print(f"Fetching data for: {datetime.now().strftime('%Y-%m-%d %H')}:00:00")

        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        # Extract current conditions
        current = data.get("currentConditions")
        if not current:
            print("No current conditions found in response.")
            return None

        # Convert to DataFrame
        df_hours = pd.DataFrame([current]) 
        
        # Select columns - check if they exist to prevent KeyErrors
        cols = ['datetime', 'temp', 'humidity']
        df_hours = df_hours[cols]

        # FIX: Ensure datetime includes today's date so localization works correctly
        today_date = datetime.now().strftime('%Y-%m-%d')
        df_hours['datetime'] = pd.to_datetime(today_date + ' ' + df_hours['datetime'])
        
        # Localize and Floor
        df_hours['datetime'] = df_hours['datetime'].dt.tz_localize('UTC').dt.floor('h')
        df_hours.rename(columns={'datetime': 'bin'}, inplace=True)

        print(f"Successfully retrieved weather data.")
        return df_hours

    except Exception as e:
        print(f"Failed to extract weather data: {e}")
        return None # Return None instead of raising to prevent Colab from hanging

In [113]:
data = extract_historical_pickup_data(start_date, end_date)
df = get_nyc_prediction_weather_data()
complete = engineer_temporal_prediction_features(df)
print(df.shape, data.shape)
complete.head()

df.head()

2026-01-19 16:01:11,896 INFO: Closing external client and cleaning up certificates.
Connection closed.
2026-01-19 16:01:11,901 INFO: Initializing external client
2026-01-19 16:01:11,905 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-19 16:01:17,128 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1257642
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (145.72s) 
Successfully retrieved 6240 rows for window: 2025-11-30 00:00:00 to 2025-12-01 00:00:00
2026-01-19 16:03:55,527 INFO: Fetching data for current timestamp 2026-01-19, 12:00:00 ...
2026-01-19 16:03:57,193 INFO: Successfully Retrieved the weather data for 2026-01-19, 12:00:00.
2026-01-19 16:03:57,196 INFO: Temporal Feature for prediction generated successfully.
(1, 5) (6240, 16)


,bin,temp,humidity,pickup_hour,is_rush_hour
0,2026-01-19 12:00:00,29.9,61.3,12,0


In [114]:
unique_pulocationids = pd.DataFrame({'pulocationid': data.pulocationid.unique()})
complete = pd.merge(unique_pulocationids, complete, how='cross')
complete.head()

,pulocationid,bin,temp,humidity,pickup_hour,is_rush_hour
0,68,2026-01-19 12:00:00,29.9,61.3,12,0
1,63,2026-01-19 12:00:00,29.9,61.3,12,0
2,109,2026-01-19 12:00:00,29.9,61.3,12,0
3,168,2026-01-19 12:00:00,29.9,61.3,12,0
4,242,2026-01-19 12:00:00,29.9,61.3,12,0


In [115]:
new_df = pd.concat([data, complete], axis=0, ignore_index=True)
new_df.shape

(6500, 16)

In [116]:
new_df.tail()

,temp,humidity,pickup_hour,is_rush_hour,city_avg_speed,zone_avg_speed,zone_congestion_index,pickups_lag_1h,pulocationid,pickups_lag_24h,city_pickups_lag_1h,neighbor_pickups_lag_1h,neighbor_pickups_sum,pickups,bin_str,bin
6495,29.9,61.3,12,0,NaN,NaN,NaN,NaN,108,NaN,NaN,NaN,NaN,NaN,NaN,2026-01-19 12:00:00
6496,29.9,61.3,12,0,NaN,NaN,NaN,NaN,198,NaN,NaN,NaN,NaN,NaN,NaN,2026-01-19 12:00:00
6497,29.9,61.3,12,0,NaN,NaN,NaN,NaN,164,NaN,NaN,NaN,NaN,NaN,NaN,2026-01-19 12:00:00
6498,29.9,61.3,12,0,NaN,NaN,NaN,NaN,166,NaN,NaN,NaN,NaN,NaN,NaN,2026-01-19 12:00:00
6499,29.9,61.3,12,0,NaN,NaN,NaN,NaN,143,NaN,NaN,NaN,NaN,NaN,NaN,2026-01-19 12:00:00


In [ ]:
new_df.sort_values(by=['pulocationid','bin'], inplace=True)
new_df.set_index(['bin', 'pulocationid'], inplace=True)
new_df.head() 

In [4]:
import os, sys
import pandas as pd

In [9]:
 #Cache neighbor dictionary
_neighbor_dict = None
_neighbor_cache_path = os.path.join('artifacts/inference', "neighbors.pkl")

def _get_neighbor_dict() -> dict:
        global _neighbor_dict
        if _neighbor_dict is not None:
            return _neighbor_dict

        if os.path.exists(_neighbor_cache_path):
            try:
                with open(_neighbor_cache_path, "rb") as f:
                    _neighbor_dict = pickle.load(f)
                print("Loaded neighbor dictionary from cache")
                return _neighbor_dict
            except Exception as e:
                print(f"Failed to load neighbor cache: {e}")

        zones_gdf = load_shapefile_from_zip(config.taxi_zone_shapefile_url,
                                            config.shapefile_dir)
        zones_gdf_left = zones_gdf.rename(columns={"LocationID": "LocationID_left"})
        zones_gdf_right = zones_gdf.rename(columns={"LocationID": "LocationID_right"})
        neighbors_df = gpd.sjoin(zones_gdf_left, zones_gdf_right, how="left", predicate="touches")
        neighbors_df = neighbors_df[neighbors_df['LocationID_left'] != neighbors_df['LocationID_right']]
        _neighbor_dict = (neighbors_df.groupby('LocationID_left')['LocationID_right']
                                .apply(lambda s: sorted(list(set(s))))
                                .to_dict())
        try:
            os.makedirs(config.shapefile_dir, exist_ok=True)
            with open(_neighbor_cache_path, "wb") as f:
                pickle.dump(_neighbor_dict, f)

        except Exception as e:
            print(f"Failed to persist neighbor cache: {e}")
        logger.info("neighbor dict retrieved successfully")
        return _neighbor_dict

In [10]:
_get_neighbor_dict()

NameError: name 'load_shapefile_from_zip' is not defined

In [2]:
from src.DynamicPricingEngine.utils.common_utils import load_shapefile_from_zip
from src.DynamicPricingEngine.config.configuration import ConfigurationManager
config = ConfigurationManager()
infconf= config.get_inference_config()

print(infconf.shapefile_dir)
print(infconf.taxi_zone_shapefile_url)

[ 2026-01-21 12:20:41,573 ]: DynamicPricingEngine: INFO: common_utils: 46: successfully load the yaml file from path: config/config.yaml
[ 2026-01-21 12:20:41,662 ]: DynamicPricingEngine: INFO: common_utils: 46: successfully load the yaml file from path: params.yaml
[ 2026-01-21 12:20:41,666 ]: DynamicPricingEngine: INFO: common_utils: 102: successfully created directory at: ['artifacts']
[ 2026-01-21 12:20:41,667 ]: DynamicPricingEngine: INFO: configuration: 28: Artifacts root directory successfully created: artifacts
[ 2026-01-21 12:20:41,670 ]: DynamicPricingEngine: INFO: common_utils: 102: successfully created directory at: ['artifacts/inference']
[ 2026-01-21 12:20:41,671 ]: DynamicPricingEngine: INFO: configuration: 113: Inference root directory created successfully: artifacts/inference
artifacts/inference/shapefile
https://d37ci6vzurychx.cloudfront.net/misc/taxi_zones.zip


In [11]:
import requests
import zipfile
import io

In [ ]:
# Load the official NYC Taxi Zone lookup table
import pandas as pd
zone_lookup_url = "https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv"
zone_df = pd.read_csv('https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv')#zone_lookup_url)

print(zone_df.head())

SyntaxError: invalid syntax (2268227159.py, line 4)

In [ ]:
def load_shapefile_from_zip(url, extract_to):
    # Step 1: Download the zipfile
    response = requests.get(url)
    response.raise_for_status()  # raise error if download failed

    # Step 2: Extract the zipfile
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        z.extractall(extract_to)

    # Step 3: Find the .shp file (main shapefile component)
    shp_file = None
    for root, dirs, files in os.walk(extract_to):
        for file in files:
            if file.endswith(".shp"):
                shp_file = os.path.join(root, file)
                break

    if shp_file is None:
        raise FileNotFoundError("No .shp file found in the extracted archive.")

    # Step 4: Load shapefile into GeoDataFrame
    gdf = gpd.read_file(shp_file)
    return gdf

In [ ]:
#gdf = load_shapefile_from_zip(infconf.taxi_zone_shapefile_url, infconf.shapefile_dir)

BadZipFile: File is not a zip file

In [9]:
response = requests.get(infconf.taxi_zone_shapefile_url)
print(response.content)

b''


In [ ]:
#zones_gdf = load_shapefile_from_zip(infconf.taxi_zone_shapefile_url,
                 #                       infconf.shapefile_dir)

BadZipFile: File is not a zip file

In [ ]:
 #Cache neighbor dictionary
_neighbor_dict = None
_neighbor_cache_path = os.path.join('artifacts/inference', "neighbors.pkl")

def _get_neighbor_dict() -> dict:
    global _neighbor_dict  # Declare it as global
    if _neighbor_dict is not None:
        return _neighbor_dict

    if os.path.exists(_neighbor_cache_path):
        try:
            with open(_neighbor_cache_path, "rb") as f:
                _neighbor_dict = pickle.load(f)
            print("Loaded neighbor dictionary from cache")
            return _neighbor_dict
        except Exception as e:
            print(f"Failed to load neighbor cache: {e}")

    zones_gdf = load_shapefile_from_zip(taxi_zone_shapefile_url,
                                        shapefile_dir)
    zones_gdf_left = zones_gdf.rename(columns={"LocationID": "LocationID_left"})
    zones_gdf_right = zones_gdf.rename(columns={"LocationID": "LocationID_right"})
    neighbors_df = gpd.sjoin(zones_gdf_left, zones_gdf_right, how="left", predicate="touches")
    neighbors_df = neighbors_df[neighbors_df['LocationID_left'] != neighbors_df['LocationID_right']]
    _neighbor_dict = (neighbors_df.groupby('LocationID_left')['LocationID_right']
                            .apply(lambda s: sorted(list(set(s))))
                            .to_dict())
    try:
        os.makedirs(shapefile_dir, exist_ok=True)
        with open(_neighbor_cache_path, "wb") as f:
            pickle.dump(_neighbor_dict, f)

    except Exception as e:
        print(f"Failed to persist neighbor cache: {e}")
    return _neighbor_dict